<center><img src="https://cdn.britannica.com/64/155864-050-34FBD7A2/view-Great-Barrier-Reef-Australia-coast.jpg" width=700></center>

<center><h1>Great Barrier Reef - YOLO5 and Detectron2 Full Understanding</h1></center>



## Introduction 

* YOLO an acronym for 'You only look once', is an object detection algorithm that divides images into a grid system. Each cell in the grid is responsible for detecting objects within itself.

* YOLO is one of the most famous object detection algorithms due to its speed and accuracy.


<center><img src="https://debuggercafe.com/wp-content/uploads/2021/03/Object-Detection-using-PyTorch-YOLOv5-e1617643548833.jpg"></center>

## YOLO5 Architecture 

In [ ]:
import torch, torchvision
print(torch.__version__, torch.cuda.is_available())  

In [ ]:
!nvidia-smi

In [ ]:
import sys
sys.path.append('../input/detectron-data/mysitepackages')

In [ ]:
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np
import pandas as pd 

from tqdm import tqdm
from tqdm import tqdm_notebook as tqdm # progress bar
from tqdm.notebook import tqdm
tqdm.pandas()

from datetime import datetime
import time
import matplotlib.pyplot as plt

import os, json, cv2, random
import skimage.io as io
import copy
from pathlib import Path
from typing import Optional
import json
import matplotlib.pyplot as plt

import ast


from tqdm import tqdm
import itertools

import torch
import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2

from glob import glob
import numba
from numba import jit

from pycocotools.coco import COCO
# detectron2
from detectron2.structures import BoxMode
from detectron2 import model_zoo
from detectron2.config import get_cfg
from detectron2.data import DatasetCatalog, MetadataCatalog
from detectron2.engine import DefaultPredictor, DefaultTrainer, launch
from detectron2.evaluation import COCOEvaluator
from detectron2.structures import BoxMode
from detectron2.utils.visualizer import ColorMode
from detectron2.utils.logger import setup_logger
from detectron2.utils.visualizer import Visualizer

from detectron2.data import DatasetCatalog, MetadataCatalog, build_detection_test_loader, build_detection_train_loader
from detectron2.data import detection_utils as utils


from detectron2.data import DatasetCatalog, MetadataCatalog, build_detection_test_loader, build_detection_train_loader
from detectron2.data import detection_utils as utils
import detectron2.data.transforms as T
from detectron2.evaluation import COCOEvaluator, inference_on_dataset



from detectron2.evaluation.evaluator import DatasetEvaluator
import pycocotools.mask as mask_util
from detectron2.engine import BestCheckpointer
from detectron2.checkpoint import DetectionCheckpointer

setup_logger()

In [ ]:
# !python -m pip install 'git+https://github.com/facebookresearch/detectron2.git' --target=/kaggle/working/mysitepackages

In [ ]:
# import sys
# sys.path.append('../input/detectron-data/mysitepackages')

In [ ]:
# Read training dataset
train_df = pd.read_csv("../input/tensorflow-great-barrier-reef/train.csv")
test_df = pd.read_csv("../input/tensorflow-great-barrier-reef/test.csv")

In [ ]:
# train_df

In [ ]:
# eval(train_df["annotations"][100])[0]

In [ ]:
# Create a "path" column containing full path to the frames
base_folder = "../input/tensorflow-great-barrier-reef/train_images"

train_df["path"] = base_folder + "/video_" + \
                    train_df['video_id'].astype(str) + "/" +\
                    train_df['video_frame'].astype(str) +".jpg"
train_df["Width"]=1280
train_df["Height"]=720

In [ ]:
train_df["NumBBox"]=train_df['annotations'].apply(lambda x: str.count(x, 'x'))
train_df.head(5)

In [ ]:
print(train_df["NumBBox"].unique())


In [ ]:
df_train=train_df[train_df["NumBBox"]>0]
df_train.reset_index(drop=True, inplace=True)
df_train

In [ ]:
df_train.shape[0]

In [ ]:
df_train["image_id"][0]

In [ ]:
print(df_train["NumBBox"].unique())


In [ ]:
def load_dataset(train_df) : 
    base_folder = "../input/tensorflow-great-barrier-reef/train_images"
    dataset_dict = []
    for index in np.arange(train_df.shape[0]) : 
        record = {}
        record["image_id"] = train_df["image_id"][index]
        record["file_name"] = train_df["path"][index]
        #record["file_name"] = f'../input/tensorflow-great-barrier-reef/train_images/video_{train_df["video_id"][index]}/{train_df["video_frame"][index]}.jpg'
        #../input/tensorflow-great-barrier-reef/train_images/video_0/0.jpg
        record["width"] = train_df["Width"][index]
        record["height"] = train_df["Height"][index]
        class_id = 0
        obj = []
        
        for box in np.arange(len(eval(train_df.annotations[index]))) : 
            annot = {}
            annot["bbox"] = [eval(train_df.annotations[index])[box]['x'],
                            eval(train_df.annotations[index])[box]['y'],
                            eval(train_df.annotations[index])[box]['width'],
                            eval(train_df.annotations[index])[box]['height']]
            annot["bbox_mode"] =  BoxMode.XYWH_ABS
            annot["category_id"] = class_id
            obj.append(annot)
        record["annotations"] = obj
        dataset_dict.append(record)
    return dataset_dict 
    

In [ ]:
# train_dataset =  load_dataset(df_train)


In [ ]:
# train_dataset

In [ ]:
# len(train_dataset)

In [ ]:
# train_dataset[90:100]

In [ ]:
# train_dataset[100]

In [ ]:
DatasetCatalog.register("train_dataset" ,lambda d=df_train: load_dataset(d)[:])
#atasetCatalog.register("test_dataset" ,lambda d=df_train: load_dataset(d)[4500:])

# #DatasetCatalog.register("test_dataset" ,lambda d=train_dataset[90:100]: load_dataset(d))


In [ ]:
from detectron2.engine import DefaultTrainer

cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml"))
cfg.DATASETS.TRAIN = ("train_dataset",)
cfg.DATASETS.TEST = ()
cfg.DATALOADER.NUM_WORKERS = 2
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml")  # Let training initialize from model zoo
cfg.SOLVER.IMS_PER_BATCH = 2
cfg.SOLVER.BASE_LR = 0.00025  # pick a good LR
cfg.SOLVER.MAX_ITER = 300    # 300 iterations seems good enough for this toy dataset; you will need to train longer for a practical dataset
cfg.SOLVER.STEPS = []        # do not decay learning rate
#cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 128   # faster, and good enough for this toy dataset (default: 512)
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1  # only has one class (ballon). (see https://detectron2.readthedocs.io/tutorials/datasets.html#update-the-config-for-new-datasets)
# NOTE: this config means the number of classes, but a few popular unofficial tutorials incorrect uses num_classes+1 here.

os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
trainer = DefaultTrainer(cfg) 
trainer.resume_or_load(resume=False)
trainer.train()

In [ ]:
# from detectron2.engine import DefaultTrainer

# cfg = get_cfg()
# cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml"))
# cfg.DATASETS.TRAIN = ("train_dataset",)
# cfg.DATASETS.TEST = ()
# #cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5 
# cfg.DATALOADER.NUM_WORKERS = 2
# cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml")  # Let training initialize from model zoo
# cfg.SOLVER.IMS_PER_BATCH = 2
# cfg.SOLVER.BASE_LR = 0.001 # pick a good LR
# cfg.SOLVER.MAX_ITER = 40000  # 300 iterations seems good enough for this toy dataset; you will need to train longer for a practical dataset
# cfg.SOLVER.GAMMA = 0.1
# # The iteration number to decrease learning rate by GAMMA.
# cfg.SOLVER.STEPS = (10000,)
# #cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 2   # faster, and good enough for this toy dataset (default: 512)
# cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1 # only has one class (ballon). (see https://detectron2.readthedocs.io/tutorials/datasets.html#update-the-config-for-new-datasets)
# # NOTE: this config means the number of classes, but a few popular unofficial tutorials incorrect uses num_classes+1 here.

# os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
# trainer = DefaultTrainer(cfg) 
# trainer.resume_or_load(resume=False)
# trainer.train()

In [ ]:
# from detectron2.engine import DefaultTrainer

# cfg = get_cfg()
# cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml"))
# cfg.DATASETS.TRAIN = ("train_dataset")
# #cfg.DATASETS.TEST = ("test_dataset")
# cfg.DATALOADER.NUM_WORKERS = 2
# #cfg.OUTPUT_DIR = '../input/detectron-data/output'

# cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml")  # Let training initialize from model zoo
# #cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")
# cfg.SOLVER.IMS_PER_BATCH = 2
# cfg.SOLVER.BASE_LR = 0.00025  # pick a good LR
# cfg.SOLVER.MAX_ITER = 100    # 300 iterations seems good enough for this toy dataset; you will need to train longer for a practical dataset
# cfg.SOLVER.STEPS = []        # do not decay learning rate
# #cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 128   # faster, and good enough for this toy dataset (default: 512)
# cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1  # only has one class (ballon). (see https://detectron2.readthedocs.io/tutorials/datasets.html#update-the-config-for-new-datasets)
# # NOTE: this config means the number of classes, but a few popular unofficial tutorials incorrect uses num_classes+1 here.
# os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
# trainer = DefaultTrainer(cfg) 
# trainer.resume_or_load(resume=False)
# trainer.train()


In [ ]:
# im = cv2.imread(train_df['path'][100])


In [ ]:
# train_dataset[100]

In [ ]:
# # Inference should use the config with parameters that are used in training
# # cfg now already contains everything we've set previously. We changed it a little bit for inference:
from detectron2.engine import DefaultTrainer

cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml"))
cfg.OUTPUT_DIR = '../input/detectron-data/output'

cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")  # path to the model we just trained
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.3# set a custom testing threshold
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1

predictor = DefaultPredictor(cfg)
# outputs = predictor(array[0])
# outputs

In [ ]:
# array = np.load('../input/tensorflow-great-barrier-reef/example_test.npy')

In [ ]:
# array.shape

In [ ]:
# train_dataset[100]

In [ ]:
# outputs = predictor(im)

# outputs

In [ ]:
# train_df['annotations'][100]

In [ ]:
# test = pd.read_csv('../input/tensorflow-great-barrier-reef/example_sample_submission.csv')

In [ ]:
# test

In [ ]:
# def pred(array): 
#     for im,i in zip(array,np.arange(array.shape[0])) : 
#         outputs = predictor(im)
#         pred_boxes = outputs["instances"].pred_boxes.tensor.cpu().numpy()
#         scores = outputs["instances"].scores.cpu().numpy()
#         test.row_num[0] =  i
#         annotations = []
#         for annot in np.arange(len(scores)): 
#             annotations.append(np.concatenate((scores[annot],pred_boxes[annot]),axis=None))

In [ ]:
# from detectron2.modeling import build_model
# cfg = get_cfg()
# cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml"))
# cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml")  # Let training initialize from model zoo

# model = build_model(cfg)
# DetectionCheckpointer(model).load('../input/detectron-data/output/model_final.pth')  # load a file, usually from cfg.MODEL.WEIGHTS
# model.eval()


In [ ]:
def pred(im): 
    outputs = predictor(im)
    array = np.array([])
    pred_boxes = outputs["instances"].pred_boxes.tensor.cpu().numpy()
    scores = outputs["instances"].scores.cpu().numpy()
    annotations = []
    for annot in np.arange(len(scores)): 
        array = np.concatenate((array,scores[annot],pred_boxes[annot][0],pred_boxes[annot][1],pred_boxes[annot][2]-pred_boxes[annot][0],
                               pred_boxes[annot][3]-pred_boxes[annot][1]),axis=None)
    return array

In [ ]:
# " ".join(list(pred(im).astype(str)))

In [ ]:
# print(outputs)

In [ ]:
# # look at the outputs. See https://detectron2.readthedocs.io/tutorials/models.html#model-output-format for specification
# print(outputs["instances"].pred_boxes.tensor.cpu().numpy())
# print(outputs["instances"].scores.cpu().numpy())

In [ ]:
# # We can use `Visualizer` to draw the predictions on the image.
# v = Visualizer(im, MetadataCatalog.get(cfg.DATASETS.TRAIN[0]))
# out = v.draw_instance_predictions(outputs["instances"].to("cpu"))
# plt.imshow(out.get_image()[:, :, ::-1])

In [ ]:
# for d in random.sample(train_dataset, 1):    
#     im = cv2.imread(d["file_name"])
#     outputs = predictor(im)  # format is documented at https://detectron2.readthedocs.io/tutorials/models.html#model-output-format
#     # look at the outputs. See https://detectron2.readthedocs.io/tutorials/models.html#model-output-format for specification
#     print(outputs["instances"].pred_classes)
#     print(outputs["instances"].pred_boxes)
#     v = Visualizer(im[:, :, ::-1],
                    
#                    scale=0.5, 
#     )
#     out = v.draw_instance_predictions(outputs["instances"].to("cpu"))
# plt.imshow(out.get_image()[:, :, ::-1])

In [ ]:
import greatbarrierreef
env = greatbarrierreef.make_env()   # initialize the environment
iter_test = env.iter_test()    # an iterator which loops over the test set and sample submission
for (pixel_array, sample_prediction_df) in iter_test:
    sample_prediction_df['annotations'] = " ".join(list(pred(pixel_array).astype(str)))  # make your predictions here
    env.predict(sample_prediction_df)   # register your predictions